In [ ]:
# ==============================================================================
# PROYECTO: CRSN Slides (3J)
# BLOQUE 0: Inicialización y Verificación de Activos
# ==============================================================================

import os
from google.colab import drive
from datetime import datetime

# 1. MONTAJE DE DRIVE
if not os.path.exists('/content/drive'):
    print("📡 Montando Google Drive...")
    drive.mount('/content/drive')

# 2. DEFINICIÓN DE RUTAS REALES (Verificadas)
BASE_DIR = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025"
FOLDER_ACTIVOS = "20260126_1155_proyecto_crsn-slides"

EXCEL_PATH = os.path.join(BASE_DIR, FOLDER_ACTIVOS, "crsn-2015-2025.xlsx")
LOGO_PATH = os.path.join(BASE_DIR, FOLDER_ACTIVOS, "crsn-logo-circulo.png")

# 3. GESTIÓN DE SALIDA (Nomenclatura PJLA)
RUN_ID = datetime.now().strftime("%Y%m%d_%H%M")
OUTPUT_ROOT = os.path.join(BASE_DIR, f"{RUN_ID}_CRSN_PROD_v3.0")
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# 4. VERIFICACIÓN OBLIGATORIA (Sello de Seguridad)
print(f"🚀 Iniciando Pipeline v3.1")
print(f"📂 Carpeta de Salida: {OUTPUT_ROOT}")

activos = {"Excel": EXCEL_PATH, "Logo": LOGO_PATH}
errores = []

for nombre, path in activos.items():
    if os.path.exists(path):
        print(f"✅ {nombre} localizado.")
    else:
        errores.append(f"❌ ERROR: {nombre} NO EXISTE en {path}")

if errores:
    for e in errores: print(e)
    raise SystemExit("🛑 PROCESO DETENIDO: Faltan archivos críticos.")
else:
    print("\n👍 Cimiento validado. Proceda al BLOQUE 1.")

🚀 Iniciando Pipeline v3.1
📂 Carpeta de Salida: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/20260127_1731_CRSN_PROD_v3.0
✅ Excel localizado.
✅ Logo localizado.

👍 Cimiento validado. Proceda al BLOQUE 1.


In [ ]:
# ==============================================================================
# PROYECTO: CRSN Slides (3J)
# BLOQUE 1: Extracción de Datos y Curación Analítica
# ROL: Lead Data Scientist - Pedro J. Lancheros
# ==============================================================================

import pandas as pd
import re

# 1. CARGA DE BASE DE DATOS
print(f"📡 Procesando base de datos: {EXCEL_PATH}")
xl = pd.ExcelFile(EXCEL_PATH)
years = [s for s in xl.sheet_names if s.isdigit()]

# Contenedores maestros
data_diezmos = []
data_siervos = []

def clean_val(v):
    """Limpieza defensiva de valores numéricos."""
    if pd.isna(v): return 0.0
    s = re.sub(r'[^\d.]', '', str(v).replace(',', ''))
    try: return float(s) if s else 0.0
    except: return 0.0

# 2. BUCLE DE EXTRACCIÓN (Sello de Seguridad PJLA)
for year in years:
    df_raw = pd.read_excel(xl, sheet_name=year, header=None)

    # --- EXTRACCIÓN DIEZMOS (Basado en ancla 'SEMANA') ---
    mask_d = df_raw.iloc[:, 0].astype(str).str.strip().str.upper() == "SEMANA"
    if mask_d.any():
        idx_d = df_raw[mask_d].index[0]
        for m_idx in range(1, 13): # Enero a Diciembre
            semanas = [clean_val(df_raw.iloc[idx_d + w, m_idx]) for w in range(1, 6) if idx_d + w < len(df_raw)]
            data_diezmos.append({'Año': int(year), 'Mes': m_idx, 'COP': sum(semanas)})

    # --- EXTRACCIÓN SIERVOS (Basado en ancla 'SIERVOS') ---
    mask_s = df_raw.iloc[:, 0].astype(str).str.contains("SIERVOS", case=False, na=False)
    if mask_s.any():
        idx_s = df_raw[mask_s].index[0]
        # Barrido de filas de ministerios (rango de seguridad 35 filas)
        for i in range(1, 35):
            row = df_raw.iloc[idx_s + i] if (idx_s + i) < len(df_raw) else None
            if row is None or (str(row[0]) == 'nan' and clean_val(row[13]) == 0): continue

            nombre_raw = str(row[0]).strip().upper()
            if any(x in nombre_raw for x in ["TOTAL", "SIERVOS", "SUMA"]) or nombre_raw == "": continue

            # Captura mensual para cálculo de promedio
            for m_idx in range(1, 13):
                valor_mensual = clean_val(row[m_idx])
                if valor_mensual > 0:
                    data_siervos.append({
                        'Ministerio': nombre_raw,
                        'Año': int(year),
                        'Mes': m_idx,
                        'Personas': valor_mensual
                    })

# 3. CONSOLIDACIÓN Y APLICACIÓN DE MAPEO OBLIGATORIO
df_d_anual = pd.DataFrame(data_diezmos).groupby('Año')['COP'].sum().reset_index()
df_s_granular = pd.DataFrame(data_siervos)

# Mapeo de Nombres (E&E y Mujer Integral)
mapeo_nombres = {
    'EJECUTIVOS': 'E&E',
    'MUJERES': 'MUJER INTEGRAL'
}
df_s_granular['Ministerio'] = df_s_granular['Ministerio'].replace(mapeo_nombres)

# Cálculo de PROMEDIO ANUAL para Siervos (Evita duplicidad por mes)
df_s_anual = df_s_granular.groupby(['Ministerio', 'Año'])['Personas'].mean().reset_index()
df_s_anual['Personas'] = df_s_anual['Personas'].round(0).astype(int)

print(f"✅ Extracción y Mapeo finalizados.")
print(f"💰 Años con datos de Diezmos: {df_d_anual['Año'].unique().tolist()}")
print(f"👥 Ministerios únicos detectados: {df_s_anual['Ministerio'].nunique()}")

📡 Procesando base de datos: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/20260126_1155_proyecto_crsn-slides/crsn-2015-2025.xlsx
✅ Extracción y Mapeo finalizados.
💰 Años con datos de Diezmos: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
👥 Ministerios únicos detectados: 28


In [ ]:
# ==============================================================================
# PROYECTO: CRSN Slides (3J)
# BLOQUE 2: Renderizado de Visualizaciones (Reflex Blue Style)
# ROL: Lead Data Scientist - Pedro J. Lancheros
# ==============================================================================

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Configuración de Estilo Global (Auditorio 1000+)
BG_COLOR = '#0B0F14'
BAR_COLOR = '#001489'
EDGE_COLOR = '#3355FF'
plt.rcParams['figure.facecolor'] = BG_COLOR

def render_crsn_slide(titulo, subtitulo, eje_x, valores, nombre_archivo):
    """Genera una slide con el estándar visual CRSN/3J."""
    fig = plt.figure(figsize=(16, 9))
    ax = fig.add_axes([0.05, 0.15, 0.9, 0.65], facecolor=BG_COLOR)

    # Eje X fijo 2015-2025 para comparativa histórica
    df_plot = pd.DataFrame({'Año': range(2015, 2026)})
    data_temp = pd.DataFrame({'Año': eje_x, 'Val': valores})
    df_final = pd.merge(df_plot, data_temp, on='Año', how='left').fillna(0)

    # Renderizado de Barras
    bars = ax.bar(df_final['Año'].astype(str), df_final['Val'],
                  color=BAR_COLOR, edgecolor=EDGE_COLOR, width=0.8, linewidth=1.5)

    # Títulos y Subtítulos
    fig.text(0.5, 0.92, titulo, fontsize=48, color='white', fontweight='bold', ha='center')
    fig.text(0.5, 0.86, subtitulo, fontsize=16, color='#888888', ha='center', style='italic')

    # Etiquetas de Datos (Solo valores mayores a 0)
    max_val = max(df_final['Val']) if max(df_final['Val']) > 0 else 1
    for bar in bars:
        h = bar.get_height()
        if h > 0:
            ax.text(bar.get_x() + bar.get_width()/2, h + (max_val * 0.02),
                    f'{int(h)}', ha='center', color='white', fontsize=26, fontweight='black')

    # Limpieza de Ejes
    for s in ['top', 'right', 'left', 'bottom']: ax.spines[s].set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.tick_params(axis='x', colors='white', labelsize=20, length=0)

    # Inserción de Logo
    if os.path.exists(LOGO_PATH):
        logo = mpimg.imread(LOGO_PATH)
        logo_ax = fig.add_axes([0.05, 0.82, 0.1, 0.1], anchor='NW', zorder=10)
        logo_ax.imshow(logo)
        logo_ax.axis('off')

    # Guardado en carpeta versionada
    plt.savefig(os.path.join(OUTPUT_ROOT, nombre_archivo), dpi=150)
    plt.close()

# --- EJECUCIÓN MASIVA ---

# 1. Slide de Diezmos (Referencia 2015 = 100)
base_val = df_d_anual['COP'].iloc[0]
df_d_anual['Indice'] = (df_d_anual['COP'] / base_val * 100).round(0).astype(int)
render_crsn_slide(
    "DIEZMOS: CRECIMIENTO HISTÓRICO",
    "(Referencia Relativa: Año 2015 = 100) | Análisis 2015-2025",
    df_d_anual['Año'], df_d_anual['Indice'], "slide_01_diezmos.png"
)

# 2. Slides de Ministerios (Promedio Mensual)
print(f"🚀 Renderizando {len(df_s_anual['Ministerio'].unique())} ministerios...")
for min_name in sorted(df_s_anual['Ministerio'].unique()):
    df_m = df_s_anual[df_s_anual['Ministerio'] == min_name]
    display_title = "EJECUTIVOS Y EMPRESARIOS" if min_name == "E&E" else min_name
    render_crsn_slide(
        f"SIERVOS: {display_title}",
        "Promedio Mensual de Participación (Conteo de Personas) | Serie 2015-2025",
        df_m['Año'], df_m['Personas'], f"slide_min_{min_name.replace(' ', '_')}.png"
    )

print(f"✅ Proceso finalizado. {len(os.listdir(OUTPUT_ROOT))} archivos generados en:")
print(f"📁 {OUTPUT_ROOT}")

🚀 Renderizando 28 ministerios...
✅ Proceso finalizado. 29 archivos generados en:
📁 /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/20260127_1731_CRSN_PROD_v3.0


In [ ]:
# ==============================================================================
# PROYECTO: CRSN Slides (3J)
# BLOQUE 3: Integración de Presentación Final (PDF)
# ROL: Lead Data Scientist - Pedro J. Lancheros
# ==============================================================================

from PIL import Image
import os

# 1. DEFINICIÓN DEL ARCHIVO DE SALIDA
PDF_OUTPUT_PATH = os.path.join(BASE_DIR, f"{RUN_ID}_CRSN_PRESENTACION_FINAL.pdf")

# 2. RECOLECCIÓN Y ORDENACIÓN DE LÁMINAS
# Buscamos todos los PNGs generados en el bloque anterior
all_files = [f for f in os.listdir(OUTPUT_ROOT) if f.endswith('.png')]

# Ordenar: 'slide_01_diezmos.png' primero, luego el resto alfabéticamente
diezmos_slide = [f for f in all_files if 'diezmos' in f]
mins_slides = sorted([f for f in all_files if 'min_' in f])

final_file_list = diezmos_slide + mins_slides

# 3. CREACIÓN DEL PDF
if final_file_list:
    print(f"📄 Empaquetando {len(final_file_list)} láminas en PDF...")

    # Abrir imágenes y convertir a RGB (requerido para PDF)
    img_objects = [Image.open(os.path.join(OUTPUT_ROOT, f)).convert('RGB') for f in final_file_list]

    # Guardar la primera y anexar las demás
    img_objects[0].save(
        PDF_OUTPUT_PATH,
        save_all=True,
        append_images=img_objects[1:]
    )

    print(f"✅ PDF generado exitosamente.")
    print(f"📂 Ubicación final: {PDF_OUTPUT_PATH}")
else:
    print("❌ ERROR: No se encontraron archivos PNG en la carpeta de salida.")

# 4. CIERRE DE SESIÓN ANALÍTICA
print("\n--- RESUMEN DE PROYECTO 3J ---")
print(f"Métrica utilizada: PROMEDIO ANUAL (Basado en Serie 2015-2025)")
print(f"Estado del entregable: LISTO PARA DESCARGA")

📄 Empaquetando 29 láminas en PDF...
✅ PDF generado exitosamente.
📂 Ubicación final: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/20260127_1731_CRSN_PRESENTACION_FINAL.pdf

--- RESUMEN DE PROYECTO 3J ---
Métrica utilizada: PROMEDIO ANUAL (Basado en Serie 2015-2025)
Estado del entregable: LISTO PARA DESCARGA


In [ ]:
# ==============================================================================
# PROYECTO: CRSN Slides (3J)
# BLOQUE 4: Inventario Físico de Archivos (Auditoría de Persistencia)
# ROL: Lead Data Scientist - Pedro J. Lancheros
# ==============================================================================

import os
import pandas as pd
from datetime import datetime

def generar_inventario(ruta_busqueda):
    """Escanea la ruta y genera una tabla detallada de archivos."""
    print(f"\n🔍 Iniciando escaneo en: {ruta_busqueda}")

    if not os.path.exists(ruta_busqueda):
        print(f"❌ ERROR: La ruta no existe físicamente.")
        return

    inventario = []

    # Caminata por el sistema de archivos
    for raiz, directorios, archivos in os.walk(ruta_busqueda):
        for nombre in archivos:
            ruta_completa = os.path.join(raiz, nombre)
            stats = os.stat(ruta_completa)

            inventario.append({
                'Carpeta': os.path.basename(raiz),
                'Archivo': nombre,
                'Tamaño (KB)': round(stats.st_size / 1024, 2),
                'Última Modif': datetime.fromtimestamp(stats.st_mtime).strftime('%Y-%m-%d %H:%M:%S')
            })

    # Crear tabla de reporte
    df_inv = pd.DataFrame(inventario)

    if not df_inv.empty:
        print(f"✅ Se encontraron {len(df_inv)} archivos.")
        # Ordenar por fecha de modificación para ver lo más reciente arriba
        return df_inv.sort_values(by='Última Modif', ascending=False)
    else:
        print("⚠️ No se encontraron archivos en esta ruta.")
        return None

# --- EJECUCIÓN DEL INVENTARIO ---

# Ruta de la última corrida (usamos la variable definida en el Bloque 0)
reporte = generar_inventario(OUTPUT_ROOT)

if reporte is not None:
    # Mostrar el inventario completo
    display(reporte)

    # Guardar el inventario en un CSV para tu cuaderno de Aurora
    inv_path = os.path.join(OUTPUT_ROOT, "inventario_activos_v3.1.csv")
    reporte.to_csv(inv_path, index=False)
    print(f"\n📄 Inventario guardado como: {inv_path}")


🔍 Iniciando escaneo en: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/20260127_1731_CRSN_PROD_v3.0
✅ Se encontraron 29 archivos.


,Carpeta,Archivo,Tamaño (KB),Última Modif
28,20260127_1731_CRSN_PROD_v3.0,slide_min_VISA.png,93.33,2026-01-27 17:31:50
27,20260127_1731_CRSN_PROD_v3.0,slide_min_UJIERES.png,107.88,2026-01-27 17:31:49
26,20260127_1731_CRSN_PROD_v3.0,slide_min_TMT.png,105.89,2026-01-27 17:31:48
25,20260127_1731_CRSN_PROD_v3.0,slide_min_TIMOTEOS.png,88.12,2026-01-27 17:31:48
24,20260127_1731_CRSN_PROD_v3.0,slide_min_ROCAFE.png,100.48,2026-01-27 17:31:47
23,20260127_1731_CRSN_PROD_v3.0,slide_min_ROCA_KIDS.png,117.21,2026-01-27 17:31:47
22,20260127_1731_CRSN_PROD_v3.0,slide_min_PART._M._OPERATIVOS.png,93.59,2026-01-27 17:31:46
21,20260127_1731_CRSN_PROD_v3.0,slide_min_ORACION.png,108.31,2026-01-27 17:31:46
20,20260127_1731_CRSN_PROD_v3.0,slide_min_NO._GRUPOS.png,91.91,2026-01-27 17:31:45
19,20260127_1731_CRSN_PROD_v3.0,slide_min_NJ.png,90.07,2026-01-27 17:31:45



📄 Inventario guardado como: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/20260127_1731_CRSN_PROD_v3.0/inventario_activos_v3.1.csv


In [ ]:
# ==============================================================================
# PROYECTO: CRSN Slides (3J)
# BLOQUE 5: Inventario Cuantitativo de Directorios
# ROL: Lead Data Scientist - Pedro J. Lancheros
# ==============================================================================

import os
import pandas as pd

def auditar_proyectos(ruta_raiz):
    """Cuenta archivos por cada subcarpeta en la ruta raíz."""
    print(f"🔍 Auditando raíz de proyectos: {ruta_raiz}\n")

    if not os.path.exists(ruta_raiz):
        print("❌ ERROR: La ruta raíz no existe.")
        return

    resumen_carpetas = []

    # Listar solo el primer nivel de carpetas dentro de 01_projects
    try:
        subcarpetas = [d for d in os.listdir(ruta_raiz) if os.path.isdir(os.path.join(ruta_raiz, d))]
    except Exception as e:
        print(f"❌ Error al acceder a la ruta: {e}")
        return

    for folder in subcarpetas:
        path_completo = os.path.join(ruta_raiz, folder)

        # Conteo total de archivos dentro de esta carpeta (incluyendo subcarpetas)
        total_archivos = 0
        for _, _, archivos in os.walk(path_completo):
            total_archivos += len(archivos)

        resumen_carpetas.append({
            'Proyecto / Carpeta': folder,
            'Cantidad de Archivos': total_archivos,
            'Ruta Física': path_completo
        })

    # Crear DataFrame para visualización limpia
    df_resumen = pd.DataFrame(resumen_carpetas)

    if not df_resumen.empty:
        # Ordenar por cantidad de archivos (descendente)
        return df_resumen.sort_values(by='Cantidad de Archivos', ascending=False)
    else:
        print("⚠️ No se encontraron subcarpetas con archivos.")
        return None

# --- EJECUCIÓN ---
PATH_MAESTRO = "/content/drive/MyDrive/01_projects/"
reporte_proyectos = auditar_proyectos(PATH_MAESTRO)

if reporte_proyectos is not None:
    # Mostrar tabla resumen
    print("📋 RESUMEN DE ARCHIVOS POR PROYECTO:")
    display(reporte_proyectos)

    # Guardar en el Drive para trazabilidad Aurora
    resumen_path = os.path.join(PATH_MAESTRO, "inventario_proyectos_3J.csv")
    reporte_proyectos.to_csv(resumen_path, index=False)
    print(f"\n✅ Reporte consolidado guardado en: {resumen_path}")

🔍 Auditando raíz de proyectos: /content/drive/MyDrive/01_projects/

📋 RESUMEN DE ARCHIVOS POR PROYECTO:


,Proyecto / Carpeta,Cantidad de Archivos,Ruta Física
7,honduras-vida_abundante,55,/content/drive/MyDrive/01_projects/honduras-vi...
1,Pymes Estrategicas,51,/content/drive/MyDrive/01_projects/Pymes Estra...
0,Escribir,40,/content/drive/MyDrive/01_projects/Escribir
8,crsn-slides-actividades-2025,38,/content/drive/MyDrive/01_projects/crsn-slides...
3,2025-10-04 Consumado es. Cerrando ciclos,21,/content/drive/MyDrive/01_projects/2025-10-04 ...
5,3J-Master,12,/content/drive/MyDrive/01_projects/3J-Master
2,Yasmin Quiroga,2,/content/drive/MyDrive/01_projects/Yasmin Quiroga
4,python-final,0,/content/drive/MyDrive/01_projects/python-final
6,01-peter_boat,0,/content/drive/MyDrive/01_projects/01-peter_boat



✅ Reporte consolidado guardado en: /content/drive/MyDrive/01_projects/inventario_proyectos_3J.csv


In [ ]:
# ==============================================================================
# PROYECTO: CRSN Slides (3J)
# BLOQUE 6: Inventario de Subcarpetas del Proyecto
# ROL: Lead Data Scientist - Pedro J. Lancheros
# ==============================================================================

import os
import pandas as pd

# Definición de la ruta específica del proyecto
PATH_CRSN = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025"

def inventario_detallado(ruta):
    print(f"🔍 Auditando subcarpetas en: {ruta}\n")

    if not os.path.exists(ruta):
        print("❌ ERROR: La ruta del proyecto no existe.")
        return

    detalles = []

    # Listar contenido (archivos y carpetas)
    items = os.listdir(ruta)

    # 1. Contar archivos en la raíz del proyecto
    archivos_raiz = [f for f in items if os.path.isfile(os.path.join(ruta, f))]
    detalles.append({
        'Ubicación / Subcarpeta': '(Raíz del Proyecto)',
        'Cantidad de Archivos': len(archivos_raiz)
    })

    # 2. Contar archivos en cada subcarpeta
    subcarpetas = [d for d in items if os.path.isdir(os.path.join(ruta, d))]

    for folder in sorted(subcarpetas):
        path_completo = os.path.join(ruta, folder)

        # Conteo recursivo de archivos
        conteo = sum([len(files) for r, d, files in os.walk(path_completo)])

        detalles.append({
            'Ubicación / Subcarpeta': folder,
            'Cantidad de Archivos': conteo
        })

    # Crear tabla de resultados
    df_detallado = pd.DataFrame(detalles)
    return df_detallado

# --- EJECUCIÓN ---
reporte_detallado = inventario_detallado(PATH_CRSN)

if reporte_detallado is not None:
    print("📋 DESGLOSE DE ARCHIVOS POR SUBFOLDER:")
    display(reporte_detallado)

    # Guardar este desglose para tu cuaderno de Aurora
    detalles_path = os.path.join(PATH_CRSN, "desglose_archivos_crsn_2025.csv")
    reporte_detallado.to_csv(detalles_path, index=False)
    print(f"\n✅ Inventario detallado guardado en: {detalles_path}")

🔍 Auditando subcarpetas en: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025

📋 DESGLOSE DE ARCHIVOS POR SUBFOLDER:


,Ubicación / Subcarpeta,Cantidad de Archivos
0,(Raíz del Proyecto),6
1,20260126_1155_proyecto_crsn-slides,2
2,20260127_1731_CRSN_PROD_v3.0,30



✅ Inventario detallado guardado en: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/desglose_archivos_crsn_2025.csv
